In [1]:
#using Markdown; Markdown.parse_file("README.md")

In [2]:
MODEL_PATH_KEY = :wNB;
const FLTYPE = Float32;
include("heavyloadings.jl");

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/65cd0904e96181bb5e062bd005a20f140d5e7005-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/65cd0904e96181bb5e062bd005a20f140d5e7005-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: MODEL_FOLDER set to C:\Users\cstamas\gdrive-csertegt\BME-GPK\Programok\ICP-models
└ @ Main C:\Users\cstamas\Documents\SZTAKI\PlayGround\SzimGeo-ICP\heavyloadings.jl:17


In [3]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [4]:
include("firsticp.jl")

sortIndandTraits

In [5]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [6]:
IJuliaCell(vis)

UndefVarError: UndefVarError: vis not defined

In [7]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

UndefVarError: UndefVarError: vis not defined

## Old workflow
Every array is using `Point3f0`

```julia
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of Point3f0
reference_array = vertices(reference_pc);
reading_array = vertices(reading_pc);
#KD tree for the reference array
#kdtree_ref = KDTree(reference_array);

#Sampling random elements from the reference array
sampled_points_ref = randomSamplePoints(reference_array,0.5);
#Building KD tree for the sampled points
kdtree_sampled_ref = KDTree(sampled_points_ref);

#pairing the points in the two meshes
indexer,fl_traits = createKnnPairArray(reading_array,kdtree_sampled_ref,sampled_points_ref,true);
#rejecting the worst 20% based on the distance
rej_is,rej_fltrs = rejectWorstPercent(indexer,fl_traits,0.2);
```

## New workflow
Using Homogeneous vector instead of `Point3f0`
### Workflow lépései:
1. [x] reading és ref mesh-t homogén koordinátás SVector/MVector**(?)** tömbbé alakítani
1. [x] a teljes referencia tömbből kd-tree
### Iteráció
1. [x] reading tömbből sampling
1. [x] samplinged tömbhöz párkeresés a fából
1. ##### Worst `x`% rejection
  * [x] párok sorbarendezése távolság szerint
  * [x] a sorbarendezett párok `x`%-nak kidobása
1. ##### 2.5sigma rejection
  * [ ] szórás számítása
  * [ ] 2.5 sigmánál nagyobbak kidobása
1. [ ] A\b összeállítása és megoldása
1. [ ] transzformáció a megoldással
1. [ ] transzformáció mentése vizualizációhoz
1. [ ] kilépési feltétel check

### Iteráció vége

11. [ ] place in MeshCat
12. [ ] animate in MeshCat


In [8]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of SVector
reference_array = convert2HomogeneousArray(vertices(reference_pc),FLTYPE);
reading_array = convert2HomogeneousArray(vertices(reading_pc),FLTYPE);
#Sampling random elements from the reference and the reading array
sampled_points_ref = randomSamplePoints(reference_array,50);
sampled_points_red = randomSamplePoints(reading_array,50);
#Building KD tree for the sampled ref points and the whole ref array
kdtree_sampled_ref = KDTree(sampled_points_ref);
kdtree_ref = KDTree(reference_array);
#pairing the points in the two meshes
indM, traitM = createKnnPairArray(sampled_points_red,kdtree_ref);
#sorting the indexes by the distances in the trait matrix
s_indM, s_it = sortIndexes(indM, traitM[:,2]);

┌ Info: The white noise scaling factor is 0.014079800248146057
└ @ Main C:\Users\cstamas\Documents\SZTAKI\PlayGround\SzimGeo-ICP\firsticp.jl:100
